In [40]:
import pandas as pd

In [41]:
from pymongo import MongoClient

client = MongoClient()
db = client.ecac
logs_ecac = db.logs_ecac

In [42]:
path = '/home/helder/estudos/tcc-pucmg-2/src/python/ecac/CAVFiltro-20210628.txt'
cols = ['DATAHORA', 'IP', 'TIPO', 'TIPONI', 'NI', 'EMISSOR', 'IDUSUARIO', 'TIPONIPAPEL', 'NIPAPEL', 'PAPELATUAL', 
    'ULTIMOSERVICO', 'CODERRO', 'URL', 'IDSESSAO', 'DRF_REPRESENTADO', 'DRF_REPRESENTANTE', 'SISTEMA']
df = pd.read_csv(path, delimiter='\t', header=None)
df.columns = cols

In [43]:
dicts = df.T.to_dict().values()
log_list = [d for d in dicts]

In [44]:
logs_ecac.insert_many(log_list)

In [49]:
df[df['URL'].str.contains('^.*jquery.*$')]

,DATAHORA,IP,TIPO,TIPONI,NI,EMISSOR,IDUSUARIO,TIPONIPAPEL,NIPAPEL,PAPELATUAL,ULTIMOSERVICO,CODERRO,URL,IDSESSAO,DRF_REPRESENTADO,DRF_REPRESENTANTE,SISTEMA
2,20210628 08:02:25,10.43.7.107,13.0,1.0,5.136947e+09,Autoridade Certificadora SERPRORFBv5 - Hom,20AE E98B 8844 132C CEE8 18AD,1.0,5.136947e+09,1.0,NaN,10,/eprocessocontribuinte/assets/js/jquery.min.js,1ae25dabf82c7a7e3233e2f3629f7b6b9c3b1f347c6203...,910100.0,910100.0,51.0
3,20210628 08:02:25,10.43.7.107,13.0,1.0,5.136947e+09,Autoridade Certificadora SERPRORFBv5 - Hom,20AE E98B 8844 132C CEE8 18AD,1.0,5.136947e+09,1.0,NaN,10,/eprocessocontribuinte/assets/js/jquery.spin.js,1ae25dabf82c7a7e3233e2f3629f7b6b9c3b1f347c6203...,910100.0,910100.0,51.0


In [51]:
df.groupby(['IP', 'URL']).agg(Total=('DATAHORA', 'count'))

Total
IP          URL                                                      
10.43.7.107 /eprocessocontribuinte                                  1
            /eprocessocontribuinte/                                 2
            /eprocessocontribuinte/assets/js/assinador-serp...      1
            /eprocessocontribuinte/assets/js/assinador-serp...      1
            /eprocessocontribuinte/assets/js/assinador-serp...      1
            /eprocessocontribuinte/assets/js/jquery.min.js          1
            /eprocessocontribuinte/assets/js/jquery.spin.js         1
            /eprocessocontribuinte/inline.a501d381073d2451e...      1
            /eprocessocontribuinte/scripts.e3c3f0d654f67cad...      1

In [58]:
list(logs_ecac.aggregate( 
    [ 
        { '$match': { 'IP': '10.43.7.107'} },
        { '$group': { '_id': ['$IP', '$URL'], 'Total': { '$sum': 1 } } },
        { '$sort': { 'Total': -1 } }
    ] 
))

[{'_id': ['10.43.7.107', '/eprocessocontribuinte/'], 'Total': 2},
 {'_id': ['10.43.7.107',
   '/eprocessocontribuinte/assets/js/assinador-serpro/is.min.js'],
  'Total': 1},
 {'_id': ['10.43.7.107',
   '/eprocessocontribuinte/assets/js/assinador-serpro/serpro-signer-client.js'],
  'Total': 1},
 {'_id': ['10.43.7.107',
   '/eprocessocontribuinte/inline.a501d381073d2451e4ac.bundle.js'],
  'Total': 1},
 {'_id': ['10.43.7.107',
   '/eprocessocontribuinte/scripts.e3c3f0d654f67cad9988.bundle.js'],
  'Total': 1},
 {'_id': ['10.43.7.107', '/eprocessocontribuinte/assets/js/jquery.spin.js'],
  'Total': 1},
 {'_id': ['10.43.7.107',
   '/eprocessocontribuinte/assets/js/assinador-serpro/serpro-signer-promise.js'],
  'Total': 1},
 {'_id': ['10.43.7.107', '/eprocessocontribuinte'], 'Total': 1},
 {'_id': ['10.43.7.107', '/eprocessocontribuinte/assets/js/jquery.min.js'],
  'Total': 1}]